In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # Import tqdm

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Define transformations for the input data
# Adjust the size for InceptionResNet's input requirements
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Adjusting to InceptionResNet size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Load and transform datasets
train_data = ImageFolder(root='W:/OneDrive - kaist.ac.kr/KAIST/Courses/5th Sem/CS470/New/train', transform=transform)
val_data = ImageFolder(root='W:/OneDrive - kaist.ac.kr/KAIST/Courses/5th Sem/CS470/New/validation', transform=transform)
test_data = ImageFolder(root='W:/OneDrive - kaist.ac.kr/KAIST/Courses/5th Sem/CS470/New/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

class InceptionResnetA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionResnetA, self).__init__()
        self.branch1x1 = BasicConv2d(in_channels, 32, kernel_size=1)

        self.branch5x5_1 = BasicConv2d(in_channels, 32, kernel_size=1)
        self.branch5x5_2 = BasicConv2d(32, 32, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = BasicConv2d(in_channels, 32, kernel_size=1)
        self.branch3x3dbl_2 = BasicConv2d(32, 48, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = BasicConv2d(48, 64, kernel_size=3, padding=1)

        self.conv2d = nn.Conv2d(128, in_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        outputs = [branch1x1, branch5x5, branch3x3dbl]
        outputs = torch.cat(outputs, 1)
        outputs = self.conv2d(outputs)
        outputs = self.relu(outputs + x)
        return outputs

# Similarly, define InceptionResnetB and InceptionResnetC for other blocks

class InceptionResNetV1(nn.Module):
    def __init__(self, num_classes=1000):
        super(InceptionResNetV1, self).__init__()
        # Define the stem of the network
        self.stem = nn.Sequential(
            BasicConv2d(3, 32, kernel_size=3, stride=2),
            BasicConv2d(32, 32, kernel_size=3),
            BasicConv2d(32, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(3, stride=2),
            BasicConv2d(64, 80, kernel_size=1),
            BasicConv2d(80, 192, kernel_size=3),
            nn.MaxPool2d(3, stride=2)
        )

        # Define the Inception-ResNet blocks
        self.inception_resnet_a = InceptionResnetA(192)
        # Add more blocks here...

        # Define the classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(192, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.inception_resnet_a(x)
        # Pass through additional blocks...

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Create the model instance
model = InceptionResNetV1(num_classes=len(train_data.classes))


class InceptionResnetB(nn.Module):
    def __init__(self, in_channels):
        super(InceptionResnetB, self).__init__()
        self.branch7x7 = nn.Sequential(
            BasicConv2d(in_channels, 128, kernel_size=1),
            BasicConv2d(128, 128, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(128, 192, kernel_size=(7, 1), padding=(3, 0))
        )

        self.conv2d = nn.Conv2d(192, in_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        branch7x7 = self.branch7x7(x)

        outputs = self.conv2d(branch7x7)
        outputs = self.relu(outputs + x)
        return outputs

class InceptionResnetC(nn.Module):
    def __init__(self, in_channels):
        super(InceptionResnetC, self).__init__()
        self.branch3x3 = nn.Sequential(
            BasicConv2d(in_channels, 192, kernel_size=1),
            BasicConv2d(192, 224, kernel_size=(1, 3), padding=(0, 1)),
            BasicConv2d(224, 256, kernel_size=(3, 1), padding=(1, 0))
        )

        self.conv2d = nn.Conv2d(256, in_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        branch3x3 = self.branch3x3(x)

        outputs = self.conv2d(branch3x3)
        outputs = self.relu(outputs + x)
        return outputs


In [ ]:
class InceptionResNetV1(nn.Module):
    def __init__(self, num_classes=1000):
        super(InceptionResNetV1, self).__init__()
        # Define the stem of the network
        self.stem = nn.Sequential(
            BasicConv2d(3, 32, kernel_size=3, stride=2),
            BasicConv2d(32, 32, kernel_size=3),
            BasicConv2d(32, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(3, stride=2),
            BasicConv2d(64, 80, kernel_size=1),
            BasicConv2d(80, 192, kernel_size=3),
            nn.MaxPool2d(3, stride=2)
        )

        # Define the Inception-ResNet blocks
        # Adjust the number of blocks based on your requirements
        self.inception_resnet_a = nn.Sequential(*[InceptionResnetA(192) for _ in range(5)])
        self.inception_resnet_b = nn.Sequential(*[InceptionResnetB(192) for _ in range(10)])
        self.inception_resnet_c = nn.Sequential(*[InceptionResnetC(192) for _ in range(5)])

        # Define the classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(192, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.inception_resnet_a(x)
        x = self.inception_resnet_b(x)
        x = self.inception_resnet_c(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Create the model instance
model = InceptionResNetV1(num_classes=len(train_data.classes))


In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


In [ ]:
# Early Stopping
early_stopping = EarlyStopping(patience=3, min_delta=0.01)

# Training Loop
num_epochs = 5  # Set the number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(train_loader, unit="batch") as tepoch:
        for images, labels in tepoch:
            tepoch.set_description(f"Epoch {epoch+1}")
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            tepoch.set_postfix(loss=loss.item())

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad(), tqdm(val_loader, unit="batch") as vepoch:
        for images, labels in vepoch:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    print(f'Validation Loss: {val_loss}, Accuracy: {100 * correct / total}%')

    # Check early stopping condition
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

print('Finished Training')

In [ ]:
torch.save(model, '5epoch_inceptionresnetv2.pth')


In [ ]:
from sklearn.metrics import roc_curve

def calculate_eer_and_tar(y_true, y_scores, far_target=0.01):
    # Compute the ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    
    # Find the EER
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - (1 - tpr)))]
    eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]
    
    # Find the TAR at a specific FAR
    tar_at_far = tpr[np.nanargmin(np.abs(fpr - far_target))]

    return eer, tar_at_far, eer_threshold

# Use this function after obtaining y_scores (model probabilities) and y_true (actual labels)


In [ ]:
# Testing loop
model.eval()
y_scores = []
y_true = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        probabilities = F.softmax(outputs, dim=1)[:, 1]  # Probabilities for the positive class
        y_scores.extend(probabilities.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Calculate EER and TAR
eer, tar_at_far, _ = calculate_eer_and_tar(y_true, y_scores)
print(f"EER: {eer}, TAR at FAR={far_target}: {tar_at_far}")
